# Weatherpy

Conclusion
1. The temperature increases when you move towards the equator 
2. At the northern hemisphere,the temperature decreases as latitude increases
3. At the Southern hemisphere, the temperature increases as latitude increase


In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from citipy import citipy
from config import g_key
from config import api_key
import scipy.stats as st

# #import library (e.g. config.py) without restarting the kernel
# import importlib.reload(config.py)



In [ ]:
#Generate random latitudes and longitudes 
lat = []
long= []

for each_lat in np.random.randint(-90,90,1500):
    lat.append(each_lat)

for each_lon in np.random.randint(-180,180,1500):
    long.append(each_lon)

    
#Create and variable called latitude_and_Longitude and store all random latitudes and longitudes generated

latitude_and_Longitude = tuple(zip(lat,long))


In [ ]:
#output 
output_data_file = "cities_data.csv"
#use citipy library to find the nearest city for all latitudes and longitudes

cities = []
countries = []

for lat,long in latitude_and_Longitude:
    city = citipy.nearest_city(lat,long)
  

    # Replace spaces with %20 to create url correctly 
    #city = city.replace(" ", "%20")
    cityName = city.city_name
    # If the city is unique, then add it to a our cities list
    if city not in cities:

        cities.append(cityName)
        
        countries.append(city.country_code)

# Print the city counts
len(cities)
    
print('There are {} different cities using random numbers.'.format(len(set(cities))))


In [ ]:
#create a dataframe to store all the cities and countries generated with citipy

df_countries = pd.DataFrame({'Cities': cities,
              'Country': countries})

#add additional blank columns to store information from openweathermap api

df_countries['Latitude'] = ''
df_countries['Longitude'] = ''
df_countries['Temperature (F)'] = ''
df_countries['Humidity (%)'] = ''
df_countries['Cloudiness (%)'] = ''
df_countries['Wind Speed (mph)'] = ''

#drop all duplicates in column Cities.

df_countries = df_countries.drop_duplicates(subset=['Cities'],keep='first')
df_countries

In [ ]:
#Add data to dataframe 'Latitude',Longitude','Temperature (F),'Humidity (%)','Cloudiness,'Wind Speed (mph)'

for index,row in df_countries.iterrows():
    city_name = row['Cities']
    Country_id = row['Country']
    
    
    #api call for all Cities 
    api_url = "http://api.openweathermap.org/data/2.5/weather" \
    "?q={},{}&units=IMPERIAL&mode=json&APPID={}".format(city_name,Country_id,api_key)
    
    print(api_url)
    
    country_info = requests.get(api_url).json()
    
    #print all api url's for each city
    #Processing Record 21 of Set 2 | isangel
    print("Processing Record", index, "City Name: ", city_name)
    
    
    try:
        df_countries.set_value(index,'Latitude',country_info['coord']['lat'])
        df_countries.set_value(index,'Longitude',country_info['coord']['lon'])
        df_countries.set_value(index,'Temperature (F)',country_info['main']['temp_max'])
        df_countries.set_value(index,'Humidity (%)',country_info['main']['humidity'])
        df_countries.set_value(index,'Cloudiness (%)',country_info['clouds']['all'])
        df_countries.set_value(index,'Wind Speed (mph)',country_info['wind']['speed'])
        
    except KeyError:
        df_countries.set_value(index,'Latitude',np.nan)
        df_countries.set_value(index,'Longitude',np.nan)
        df_countries.set_value(index,'Temperature (F)',np.nan)
        df_countries.set_value(index,'Humidity (%)',np.nan)
        df_countries.set_value(index,'Cloudiness (%)',np.nan)
        df_countries.set_value(index,'Wind Speed (mph)',np.nan)
        
        print('Missing weather information...skip')
    

In [ ]:
# convert openweathermap api data to numerical data
df_countries['Latitude'] = pd.to_numeric(df_countries['Latitude'])
df_countries['Longitude'] = pd.to_numeric(df_countries['Longitude'])
df_countries['Temperature (F)'] = pd.to_numeric(df_countries['Temperature (F)'])
df_countries['Humidity (%)'] = pd.to_numeric(df_countries['Humidity (%)'])
df_countries['Cloudiness (%)'] = pd.to_numeric(df_countries['Cloudiness (%)'])
df_countries['Wind Speed (mph)'] = pd.to_numeric(df_countries['Wind Speed (mph)'])

#Create DataFrame and export into a .csv.
df_countries.to_csv(output_data_file,index=False)  #THIS WILL BE IN DIRECTORY
#Display the DataFrame
df_countries.head()


In [ ]:
#Latitude vs. Temperature Plot
plt.scatter(df_countries['Latitude'], df_countries['Temperature (F)'],color='g')

plt.xlabel('Latitude')
plt.ylabel('Temperature (F)')
plt.title('Latitude vs. Temperature(f)')
plt.savefig("CityLatitudeVsTemperature.png")
#conclusion
#The code is analysing the relationship between latitude and temperature
#The temperature increases when you move towards the equator 


In [ ]:
#Latitude vs. Humidity Plot
plt.scatter(df_countries['Latitude'], df_countries['Humidity (%)'],color='g')

plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.title('Latitude vs. Humidity (%)')
plt.savefig("CityLatitudeVsHumidity.png")

#Code is analyzing the relationship between temperature and humidity.


In [ ]:
#Latitude vs. Cloudiness Plot

plt.scatter(df_countries['Latitude'], df_countries['Cloudiness (%)'],color='g')

plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')
plt.title('Latitude vs. Cloudiness (%)')
plt.savefig("CityLatitudeVsCloudiness.png")

#Code is analyzing the relationship between temperature and cloudiness 


In [ ]:
plt.scatter(df_countries['Latitude'], df_countries['Wind Speed (mph)'],color='g')

plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')
plt.title('Latitude vs. Wind Speed (mph)')
plt.savefig("CityLatitudeVsWindSpeed.png")

#Code is analyzing the relationship between temperature and windspeed

In [ ]:
# OPTIONAL: Create a function for Linear Regression plots

def linear_agression(x,y):
    print(f"The r-squared is : {round(st.pearsonr(x, y)[0],2)}")
    (slope, intercept, rvalue, pvalue, stderr) = st.linregress(x, y)
    regress_values = x * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    plt.scatter(x, y)
    plt.plot(x,regress_values,"r-")
    return line_eq

# Define a fuction for annotating
def annotate(line_eq, a, b):
    plt.annotate(line_eq,(a,b),fontsize=15,color="green")

In [ ]:

#Northern Hemisphere - Temp vs. Latitude Linear Regression
#Your next objective is to run linear regression on each relationship, 
#only this time separating them into Northern Hemisphere 
#greater than or equal to 0 degrees latitude) and Southern Hemisphere (less than 0 degrees latitude):

df_countries_North = df_countries[df_countries["Latitude"] >= 0]
df_countries_South = df_countries[df_countries["Latitude"] < 0]

In [ ]:
equation = linear_agression(df_countries_North["Latitude"], df_countries_North["Temperature (F)"])

# Call an function #2
annotate(equation, 0, 0)

# Set a title
plt.title("Northern Hemisphere - Max Temp vs. Latitude Linear Regression")

# Set xlabel
plt.xlabel("Latitude")

# Set ylabel
plt.ylabel("Temp (F)")

# Save the figure
plt.savefig("Northern Hemisphere - Temp vs. Latitude Linear Regression.png")

plt.xticks(size=12)
plt.yticks(size=12)

# The correlation coefficient is a statistical measure that calculates the strength of the relationship 
# between the relative movements of two variables. The values range between -1.0 and 1.0. 
# A calculated number greater than 1.0 or less than -1.0 means that there was an error in 
# the correlation measurement. A correlation of -1.0 shows a perfect negative correlation, while a 
# correlation of 1.0 shows a perfect positive correlation. A correlation of 0.0 shows no relationship between
# the movement of the two variables.

#conclusion
#At the northern hemisphere,the temperature decreases as latitude increases

In [ ]:
# Call an function #1
equation = linear_agression(df_countries_South["Latitude"],df_countries_South["Temperature (F)"])

# Call an function #2
annotate(equation, -30, 10)

# Set a title
plt.title("Southern Hemisphere - Max Temp vs. Latitude Linear Regression")

# Set xlabel
plt.xlabel("Latitude")

# Set ylabel
plt.ylabel("Max Temp (F)")

# Save the figure
plt.savefig("Southern Hemisphere - Max Temp vs. Latitude Linear Regression.png")

plt.xticks(size=12)
plt.yticks(size=12)
#Conclusion
#At the Southern hemisphere, the temperature increases when latitude increase

In [ ]:
#Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression
# Call an function #1
equation = linear_agression(df_countries_North["Latitude"], df_countries_North["Humidity (%)"])

# Call an function #2
annotate(equation, 40, 15)

# Set a title
plt.title("Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression")

# Set xlabel
plt.xlabel("Latitude")

# Set ylabel
plt.ylabel("Humidity (%)")

# Save the figure
plt.savefig("Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression.png")
#Conclusion
#In the Northern hemisphere, Humidity increases as latitude increases 

In [ ]:
#Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

# Call an function #1
equation = linear_agression(df_countries_South["Latitude"], df_countries_South["Humidity (%)"])

# Call an function #2
annotate(equation, -40, 50)

# Set title
plt.title("Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression")

# Set xlabel
plt.xlabel("Latitude")

# Set ylabel
plt.ylabel("Humidity (%)")

# Save the figure
plt.savefig("Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression.png")
#Conclusion
#In the southern hemisphere, Humidity increases as latitude increases 

In [ ]:
#Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

equation = linear_agression(df_countries_North["Latitude"], df_countries_North["Cloudiness (%)"])

# Call an function #2
annotate(equation, 30, 40)

# Set a title
plt.title("Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression")

# Set xlabel
plt.xlabel("Latitude")

# Set ylabel
plt.ylabel("Cloudiness (%)")

# Save the figure
plt.savefig("Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression.png")

In [ ]:


# Call an function #1
equation = linear_agression(df_countries_South["Latitude"], df_countries_South["Cloudiness (%)"])

# Call an function #2
annotate(equation, -30, 40)

# Set a title
plt.title("Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression")

# Set xlabel
plt.xlabel("Latitude")

# Set ylabel
plt.ylabel("Cloudiness (%)")

# Save the figure
plt.savefig("Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression.png")

In [ ]:
#Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression
# Call an function #1
equation = linear_agression(df_countries_North["Latitude"], df_countries_North["Wind Speed (mph)"])

# Call an function #2
annotate(equation, 40, 20)

# Set  title
plt.title("Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression")

# Set xlabel
plt.xlabel("Latitude")

# Set ylabel
plt.ylabel("Wind Speed (mph)")

# Save the figure
plt.savefig("Northern Hemisphere - Wind Speed vs. Latitude Linear Regression.png")

In [ ]:
#Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression
# Call an function #1
equation = linear_agression(df_countries_South["Latitude"], df_countries_South["Wind Speed (mph)"])

# Call an function #2
annotate(equation, -30, 5)

# Set  title
plt.title("Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression")

# Set xlabel
plt.xlabel("Latitude")

# Set ylabel
plt.ylabel("Wind Speed (mph)")

# Save the figure
plt.savefig("Southern Hemisphere - Wind Speed vs. Latitude Linear Regression.png")